[produce_kitchen_station_tables.ipynb] instruction

This is our second step code. The input files needed for running this file is [Building and Kitchen List.xlsx] and [Bottle Fillers Inventory - ongoing.
xlsx] in [original_data] folder , which are original Kitchen and Station data given by client.


In this [produce_kitchen_station_tables.ipynb], we do data  Filter and Processing for original Kitchen and Station data, includes:

1. For existed water stations, we combine address A and B into a full building description and retain only [Floor,Building Description,Space@Bu room,Quantity,Date Installed,Type]. 
2. For the kitchens, we retain only [ Floor, Building, Code,Building Description,Room #]. 



After running this code, we can get [Kitchens.csv] and [Stations.csv] in [new_data] folder as result data

In [1]:
import pandas as pd

In [2]:
stations_df = pd.read_excel('../original_data/Bottle Fillers Inventory - ongoing.xlsx', sheet_name='CRC')
#stations_df = pd.read_excel('Bottle Fillers Inventory - ongoing.xlsx', sheet_name='CRC')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
stations_df.columns

Index(['Address A', 'Address B', 'Floor ', 'Description', 'Accessibility',
       'Handicap Accessible', 'Space@Bu room', 'Quantity', 'Date Installed',
       'Type', 'Walk-Thru Notes', 'Completed'],
      dtype='object')

In [4]:
# process the bottle filling stations
cleaned_stations_df = pd.DataFrame()

# first map the floor names to a number
station_floor_mapping = {
    'First': 1, 'Second': 2, 'Third': 3, 'Ninth': 9, 'Basement': -1, 'Fourth': 4, 'Fifth': 5, 
    'Sixth': 6, 'Eighth': 8, 'Seventh': 7, 'Tenth': 10, 'Eleventh': 11, 'Twelfth': 12, 'Thirteenth': 13, 
    'Fourteenth': 14, 'Fifteenth': 15, 'Sixteenth': 16, 'Seventeenth': 17, 'L1': -1, 'Production Area': 1,
    'Lobby': 1, 'Ground': 1, 'Pool': -1
}

cleaned_stations_df['Floor'] = stations_df['Floor '].str.title().str.strip().map(station_floor_mapping)

In [5]:
# construct the full address
cleaned_stations_df['Building Description'] = stations_df['Address A'].astype(str) + ' ' + stations_df['Address B'].str.upper()

In [6]:
# get the space@bu.edu, quantity, install date, and station type
keep_columns = ['Space@Bu room', 'Quantity', 'Date Installed', 'Type']
for keep_column in keep_columns:
    cleaned_stations_df[keep_column] = stations_df[keep_column]

In [7]:
# manually add each individual floor in Warren
cleaned_stations_df = pd.concat([
    cleaned_stations_df,
    pd.DataFrame({
        'Floor': list(range(5, 19)), 
        'Building Description': 14 * ['700 COMMONWEALTH AVE'],
        'Space@Bu room': None,
        'Quantity': 2, # I'm guessing 2 here, because we split 30 among 14 floors so 2 is the closest integer
        'Date Installed': 2022,
        'Type': 'Tabletop Bottle Filler'
    })
], ignore_index=True)

In [8]:
# drop rows that don't have a floor (these have been properly cleaned above already), 
# and sort by building description
cleaned_stations_df = cleaned_stations_df[~cleaned_stations_df['Floor'].isna()]
cleaned_stations_df.sort_values('Building Description', inplace=True)

# cast Floor and Quantity to int
cleaned_stations_df['Floor'] = cleaned_stations_df['Floor'].astype(int)
cleaned_stations_df['Quantity'] = cleaned_stations_df['Quantity'].astype(int)

In [9]:
cleaned_stations_df

,Floor,Building Description,Space@Bu room,Quantity,Date Installed,Type
131,1,1 UNIVERSITY RD,190A,1,NaN,NaN
205,1,1 UNIVERSITY RD,NaN,1,NaN,NaN
206,3,1 UNIVERSITY RD,390,1,NaN,NaN
132,2,1 UNIVERSITY RD,284,1,NaN,NaN
0,1,100 ASHFORD ST,191,1,NaN,Combo Bottle Filler/Drinking Fountain
...,...,...,...,...,...,...
170,3,928 COMMONWEALTH AVE,NaN,1,NaN,NaN
169,2,928 COMMONWEALTH AVE,291,1,NaN,NaN
168,1,928 COMMONWEALTH AVE,NaN,1,NaN,NaN
198,1,96 CUMMINGTON MALL,198,1,NaN,NaN


In [10]:
# convert ST to STREET, AVE to AVENUE, and RD to ROAD
def expand_end_abbreviation(building_description):
    tokens = building_description.split()
    end = tokens[-1]
    if end == 'RD':
        end = 'ROAD'
    elif end == 'ST':
        end = 'STREET'
    elif end == 'AVE':
        end = 'AVENUE'
    return ' '.join(tokens[:-1] + [end])

# s=set()
# for bd in cleaned_stations_df['Building Description'].apply(expand_end_abbreviation):
#     for token in bd.split():
#         s.add(token)
# s
cleaned_stations_df['Building Description'] = cleaned_stations_df['Building Description'].apply(expand_end_abbreviation)

In [11]:
# export to csv
cleaned_stations_df.to_csv('../new_data/Stations.csv', index=False)

In [12]:
kitchens_df = pd.read_excel('../original_data/Building and Kitchen List.xlsx', sheet_name='Rooms with Kitchens')
#kitchens_df = pd.read_excel('Building and Kitchen List.xlsx', sheet_name='Rooms with Kitchens')

In [13]:
kitchens_df.columns

Index(['Site Code (SITECODE)', 'Building Code (BLDGCODE)',
       'Building Description (BLDGDESC)', 'Building Type (BLDGTYPE)',
       'Floor (FLOORCODE)', 'Room # (RMID)', 'Cost Object Code (GROUP_)',
       'Cost Object Name (GROUPDESC)', 'FICM Code (SPACETYPE)',
       'FICM Name (SPACETYPDS)', 'Area (AREA)', 'Room Name (RMNAME)',
       'Room Note (RM_NOTE)', 'Classroom Use Extensions (EXTCLASS)',
       'Office Facilities Room Use Extensions (EXTOFFICE)',
       'General Use Facilities Room Use Extensions (EXTGENERAL)',
       'Residential Facilities Room Use Extensions (EXTRESID)'],
      dtype='object')

In [14]:
def kitchen_floor_cleaner(floor):
    if any(c.isnumeric() for c in floor):
        return str(int(''.join([c for c in floor if c.isnumeric()])))
    elif floor in ['B', 'BB', 'L1']:
        return '-1'
    elif floor in ['G', 'G1']:
        return '1'
    return floor

In [15]:
# process the kitchens
cleaned_kitchens_df = pd.DataFrame()

# first clean the floor names
cleaned_kitchens_df['Floor'] = kitchens_df['Floor (FLOORCODE)'].astype(str).str.strip().apply(kitchen_floor_cleaner)

In [16]:
# keep the bulding code, description (address), and room #
cleaned_kitchens_df['Building Code'] = kitchens_df['Building Code (BLDGCODE)']
cleaned_kitchens_df['Building Description'] = kitchens_df['Building Description (BLDGDESC)'].str.strip()
cleaned_kitchens_df['Room #'] = kitchens_df['Room # (RMID)']

In [17]:
# sort by building description and export to csv
cleaned_kitchens_df.sort_values('Building Description', inplace=True)
cleaned_kitchens_df.to_csv('../new_data/Kitchens.csv', index=False)